登入測試

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# 處理敏感資訊
from dotenv import load_dotenv
import os

load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

options = Options()
# 無頭
# options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--lang=zh-TW")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.get("https://member.tw.coupang.com/login/login.pang")
# 等待兩秒，確保頁面加載完成
time.sleep(2)

# 輸入帳號
email_input = driver.find_element(
    By.ID, "login-email-input"
)
email_input.send_keys(EMAIL)

# 輸入密碼
password_input = driver.find_element(
    By.ID, "login-password-input"
)
password_input.send_keys(PASSWORD)

# 點擊登入按鈕
login_button = driver.find_element(
    By.CLASS_NAME, "login__button--submit"
)
login_button.click()

# 等待跳轉
time.sleep(5)

# 可自行接著進入訂單頁、商品頁
print("✅ 登入流程已觸發")
# 結束時再關閉瀏覽器
# driver.quit()

登入後查詢

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
from dotenv import load_dotenv
import os

# 敏感資訊
load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

options = Options()
# 無頭模式
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--lang=zh-TW")

# 共用 driver（登入後重用）
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# === 登入流程 ===
driver.get("https://member.tw.coupang.com/login/login.pang")
time.sleep(2)

email_input = driver.find_element(
    By.ID, "login-email-input"
)
email_input.send_keys(EMAIL)

password_input = driver.find_element(
    By.ID, "login-password-input"
)
password_input.send_keys(PASSWORD)

login_button = driver.find_element(
    By.CLASS_NAME, "login__button--submit"
)
login_button.click()
time.sleep(5)

print("✅ 已登入 Coupang")


def get_coupang_search_results(driver, search_keyword: str, advanced_keywords: list[str]):
    encoded_keyword = quote(search_keyword)
    url = f"https://www.tw.coupang.com/search?q={encoded_keyword}&channel=user"
    driver.get(url)
    time.sleep(5)

    html = driver.page_source

    # 若需重複操作瀏覽器可先保留不關閉
    # driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    product_cards = soup.select(
        "div.SearchResult_searchResultProduct___h6E9"
    )

    results = []
    matched_results = []

    for card in product_cards:
        try:
            full_text = card.get_text(separator=" ", strip=True)

            title_tag = card.select_one("div.Product_title__8K0xk")
            title = title_tag.get_text(strip=True) if title_tag else "N/A"

            price_tag = card.select_one("span.Product_salePricePrice__2FbsL span")
            price = price_tag.get_text(strip=True) if price_tag else "N/A"

            unit_price_tag = card.select_one("div.Product_unitPrice__QQPdR")
            unit_price = unit_price_tag.get_text(strip=True) if unit_price_tag else "N/A"

            product = {
                "title": title,
                "full_text": full_text,
                "price": price,
                "unit_price": unit_price
            }

            results.append(product)

            if all(kw in full_text for kw in advanced_keywords):
                matched_results.append(product)

        except Exception as e:
            print("解析錯誤：", e)
            continue

    return results, matched_results


if __name__ == "__main__":
    # 關鍵字搜尋
    search_keyword = "uno 洗面乳"
    # 針對結果進階篩選
    advanced_keywords = []

    all_products, filtered_products = get_coupang_search_results(
        driver,
        search_keyword, 
        advanced_keywords
    )

    # 輸出關鍵字
    print(f"搜尋關鍵字：{search_keyword}")

    print(f"結果：{len(all_products)}")
    # 完整查詢結果
    for idx, product in enumerate(all_products, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)

    # 進階查詢
    # print(f"\n進階條件符合項目（包含：{'、'.join(advanced_keywords)}）：")
    # for idx, product in enumerate(filtered_products, 1):
    #     print(f"{idx}. 標題: {product['title']}")
    #     print(f"   價格: {product['price']}")
    #     print(f"   每單位: {product['unit_price']}")
    #     print(f"   完整內容: {product['full_text']}")
    #     print("-" * 60)
    
    driver.quit()

✅ 已登入 Coupang
搜尋關鍵字：uno 洗面乳
結果：30
1. 標題: uno 新炭洗顏乳, 130g, 2條
   價格: $126
   每單位: ($4.85/10g)
   完整內容: uno 新炭洗顏乳, 130g, 2條 特價 42折 $300 $126 ($4.85/10g) 明天 5/14 (三) 預計送達 免運 ( 1,683 )
------------------------------------------------------------
2. 標題: uno 新淨洗顏乳, 130g, 2條
   價格: $130
   每單位: ($5.00/10g)
   完整內容: uno 新淨洗顏乳, 130g, 2條 44折 $300 $130 ($5.00/10g) 明天 5/14 (三) 預計送達 免運 ( 734 )
------------------------------------------------------------
3. 標題: uno 新潤洗顏乳, 130g, 3條
   價格: $189
   每單位: ($4.85/10g)
   完整內容: uno 新潤洗顏乳, 130g, 3條 42折 $450 $189 ($4.85/10g) 明天 5/14 (三) 預計送達 免運 ( 275 )
------------------------------------------------------------
4. 標題: uno 炭洗顏控油洗面乳, 130g, 2條
   價格: $130
   每單位: ($5.00/10g)
   完整內容: uno 炭洗顏控油洗面乳, 130g, 2條 26折 $500 $130 ($5.00/10g) 明天 5/14 (三) 預計送達 免運 ( 15 )
------------------------------------------------------------
5. 標題: uno 新炭洗顏乳, 130g, 3條
   價格: $189
   每單位: ($4.85/10g)
   完整內容: uno 新炭洗顏乳, 130g, 3條 特價 42折 $450 $189 ($4.85/10g) 明天 5/14 (三) 預計送達 免運 ( 1,683

優化代碼，若進階搜尋為空列表，無需顯示進階搜尋結果

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
from dotenv import load_dotenv
import os

# 敏感資訊
load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

options = Options()
# 無頭模式
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--lang=zh-TW")

# 共用 driver（登入後重用）
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# === 登入流程 ===
driver.get("https://member.tw.coupang.com/login/login.pang")
time.sleep(2)

email_input = driver.find_element(
    By.ID, "login-email-input"
)
email_input.send_keys(EMAIL)

password_input = driver.find_element(
    By.ID, "login-password-input"
)
password_input.send_keys(PASSWORD)

login_button = driver.find_element(
    By.CLASS_NAME, "login__button--submit"
)
login_button.click()
time.sleep(5)

print("✅ 已登入 Coupang")


def get_coupang_search_results(driver, search_keyword: str, advanced_keywords: list[str]):
    encoded_keyword = quote(search_keyword)
    url = f"https://www.tw.coupang.com/search?q={encoded_keyword}&channel=user"
    driver.get(url)
    time.sleep(5)

    html = driver.page_source

    # 若需重複操作瀏覽器可先保留不關閉
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    product_cards = soup.select(
        "div.SearchResult_searchResultProduct___h6E9"
    )

    results = []
    matched_results = []

    for card in product_cards:
        try:
            full_text = card.get_text(separator=" ", strip=True)

            title_tag = card.select_one("div.Product_title__8K0xk")
            title = title_tag.get_text(strip=True) if title_tag else "N/A"

            price_tag = card.select_one("span.Product_salePricePrice__2FbsL span")
            price = price_tag.get_text(strip=True) if price_tag else "N/A"

            unit_price_tag = card.select_one("div.Product_unitPrice__QQPdR")
            unit_price = unit_price_tag.get_text(strip=True) if unit_price_tag else "N/A"

            product = {
                "title": title,
                "full_text": full_text,
                "price": price,
                "unit_price": unit_price
            }

            results.append(product)

            if all(kw in full_text for kw in advanced_keywords):
                matched_results.append(product)

        except Exception as e:
            print("解析錯誤：", e)
            continue

    return results, matched_results


if __name__ == "__main__":
    # 關鍵字搜尋
    search_keyword = "uno 洗顏乳"
    # 針對結果進階篩選
    advanced_keywords = []

    all_products, filtered_products = get_coupang_search_results(
        driver,
        search_keyword, 
        advanced_keywords
    )

    driver.quit()
    # 輸出關鍵字
    print(f"搜尋關鍵字：{search_keyword}")

    # 完整查詢結果
    for idx, product in enumerate(all_products, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)

    # 進階查詢
    print(f"\n進階條件符合項目（包含：{'、'.join(advanced_keywords)}）：")
    for idx, product in enumerate(filtered_products, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)

登入後查詢並寫入資料庫，同時顯示結果

In [ ]:
import os
import time
from datetime import datetime
from urllib.parse import quote
from dotenv import load_dotenv
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# === 環境變數與設定 ===
load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# === 建立資料庫表格與寫入 ===
def insert_into_db(data_list):
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4'
    )

    with connection:
        with connection.cursor() as cursor:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS coupang_products (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    search_keyword VARCHAR(255),
                    title TEXT,
                    full_text TEXT,
                    price VARCHAR(50),
                    unit_price VARCHAR(50),
                    timestamp DATETIME
                );
            """)
            for data in data_list:
                cursor.execute("""
                    INSERT INTO coupang_products (search_keyword, title, full_text, price, unit_price, timestamp)
                    VALUES (%s, %s, %s, %s, %s, %s);
                """, (
                    data["search_keyword"],
                    data["title"],
                    data["full_text"],
                    data["price"],
                    data["unit_price"],
                    data["timestamp"]
                ))
        connection.commit()

# === 登入 Coupang 並取得 driver ===
def login_and_get_driver():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--lang=zh-TW")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.get("https://member.tw.coupang.com/login/login.pang")
    time.sleep(2)
    driver.find_element(By.ID, "login-email-input").send_keys(EMAIL)
    driver.find_element(By.ID, "login-password-input").send_keys(PASSWORD)
    driver.find_element(By.CLASS_NAME, "login__button--submit").click()
    time.sleep(5)

    print("✅ 已登入 Coupang")
    return driver

# === 查詢商品資訊 ===
def get_coupang_search_results(driver, search_keyword: str, advanced_keywords: list[str]):
    encoded_keyword = quote(search_keyword)
    url = f"https://www.tw.coupang.com/search?q={encoded_keyword}&channel=user"
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    product_cards = soup.select("div.SearchResult_searchResultProduct___h6E9")

    now = datetime.now()
    results = []
    matched_results = []

    for card in product_cards:
        try:
            full_text = card.get_text(separator=" ", strip=True)
            title_tag = card.select_one("div.Product_title__8K0xk")
            title = title_tag.get_text(strip=True) if title_tag else "N/A"
            price_tag = card.select_one("span.Product_salePricePrice__2FbsL span")
            price = price_tag.get_text(strip=True) if price_tag else "N/A"
            unit_price_tag = card.select_one("div.Product_unitPrice__QQPdR")
            unit_price = unit_price_tag.get_text(strip=True) if unit_price_tag else "N/A"

            product = {
                "search_keyword": search_keyword,
                "title": title,
                "full_text": full_text,
                "price": price,
                "unit_price": unit_price,
                "timestamp": now
            }
            results.append(product)

            if all(kw in full_text for kw in advanced_keywords):
                matched_results.append(product)

        except Exception as e:
            print("解析錯誤：", e)
            continue

    return results, matched_results

# === 主程式 ===
if __name__ == "__main__":
    search_keyword = "uno"
    advanced_keywords = ["洗面乳"]

    driver = login_and_get_driver()
    results, matched_results = get_coupang_search_results(driver, search_keyword, advanced_keywords)
    driver.quit()

    insert_into_db(results)
    print("✅ 資料已寫入 MariaDB。\n")

    print("所有查詢結果：")
    for idx, product in enumerate(results, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)

    print(f"\n進階條件符合項目（包含：{'、'.join(advanced_keywords)}）：")
    for idx, product in enumerate(matched_results, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)
